# Task 4
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# Add any other imports you need here
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import GPT2Tokenizer, GPT2Model
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

Depending on your approach, you might need to adapt the structure of this template or parts not marked by TODOs.
It is not necessary to completely follow this template. Feel free to add more code and delete any parts that are not required.

In [2]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64  # TODO: Set the batch size according to both training performance and available memory
NUM_EPOCHS = 64  # TODO: Set the number of epochs

train_val = pd.read_csv("train.csv")
test_val = pd.read_csv("test_no_score.csv")

In [3]:
class ReviewDataset(Dataset):
    def __init__(self, embeddings_path, scores = []): 
        self.embeddings = torch.load(embeddings_path)
        self.scores = scores
    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, index):
        embeddings = self.embeddings[index]
        if len(self.scores) > 0 :
            score = self.scores[index]
            return {
                'embeddings': embeddings,
                'score': score
            }
        else:
            return {
                'embeddings': embeddings
            }

In [4]:
# Precompute embeddings
def compute_embeddings_distilbert(data_frame, tokenizer_name, max_length):
    tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_name)
    transformer = DistilBertModel.from_pretrained(tokenizer_name)
    
    embeddings = []
    for index, row in data_frame.iterrows():
        title = row[0] 
        review = row[1] 
        
        inputs = tokenizer(title + " " + review, 
                           max_length=max_length,
                           padding="max_length",
                           truncation=True,
                           return_tensors="pt")
        
        with torch.no_grad():
            outputs = transformer(**inputs)
            hidden_states = outputs.last_hidden_state
        embeddings.append(hidden_states.squeeze().cpu().numpy())
    
    return torch.tensor(embeddings)

def compute_embeddings_GPT2(data_frame, tokenizer_name, max_length):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_name)
    tokenizer.pad_token = tokenizer.eos_token
    transformer = GPT2Model.from_pretrained(tokenizer_name).to(DEVICE)
    
    embeddings = []
    for index, row in data_frame.iterrows():
        title = row[0]  
        review = row[1]  

        input = tokenizer(
            title + " " + review,
            return_tensors='pt',
            padding=True,
            truncation=True
        ).to(DEVICE)
        input_ids = input['input_ids']

        with torch.no_grad():
            outputs = transformer(input_ids)

        word_embeddings = outputs.last_hidden_state

        masked_word_embeddings = word_embeddings * input.attention_mask.unsqueeze(-1).float()

        sentence_embeddings = masked_word_embeddings.sum(dim=1)

        sentence_embeddings /= input.attention_mask.sum(dim=1, keepdim=True).float()
        
        embeddings.append(sentence_embeddings.squeeze().cpu().numpy())
    return torch.tensor(embeddings)



In [5]:
# tokenizer_name = "distilbert-base-uncased"
# max_length = 128

# train_embeddings = compute_embeddings_distilbert(train_val, tokenizer_name, max_length)
# torch.save(train_embeddings, "train_distilbert_embeddings.pt")

# test_embeddings = compute_embeddings_distilbert(test_val, tokenizer_name, max_length)
# torch.save(test_embeddings, "test_distilbert_embeddings.pt")

In [5]:
tokenizer_name = "gpt2"
max_length = 128

train_embeddings = compute_embeddings_GPT2(train_val, tokenizer_name, max_length)
torch.save(train_embeddings, "train_gpt2_embeddings.pt")

test_embeddings = compute_embeddings_GPT2(test_val, tokenizer_name, max_length)
torch.save(test_embeddings, "test_gpt2_embeddings.pt")


/home/spyros/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/tmp/ipykernel_48923/3897838643.py:69: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  return torch.tensor(embeddings)
/home/spyros/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
train_dataset = ReviewDataset("train_gpt2_embeddings.pt", train_val['score'])
test_dataset = ReviewDataset("test_gpt2_embeddings.pt")

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=16, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=16, pin_memory=True)


In [6]:
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(768, 1536)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1536, 512)
        self.sigmoid = nn.Sigmoid()
        self.fc3 = nn.Linear(512, 64)
        self.fc4 = nn.Linear(64, 1)

 
    def forward(self, x):
        
        
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x) * 10.0 
        return x



In [32]:
model = MyModule().to(DEVICE)
LEARNING_RATE = 0.001
NUM_EPOCHS = 80

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = StepLR(optimizer, step_size=1, gamma=0.88)  

model.train()
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for batch in tqdm(train_loader, total=len(train_loader)):

        embeddings = batch['embeddings'].to(DEVICE)
        scores = batch['score'].view(-1, 1).to(DEVICE)

        optimizer.zero_grad()

        outputs = model(embeddings.float()) 
        # print(outputs.shape)
        # print(scores.shape)
        loss = criterion(outputs, scores.float())

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch['embeddings'].size(0)

    scheduler.step()

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {epoch_loss:.4f}")

100%|██████████| 196/196 [00:00<00:00, 313.69it/s]


Epoch 1/80, Loss: 5.9670


100%|██████████| 196/196 [00:00<00:00, 305.25it/s]


Epoch 2/80, Loss: 3.3179


100%|██████████| 196/196 [00:00<00:00, 325.15it/s]


...

Epoch 78/80, Loss: 1.3811


100%|██████████| 196/196 [00:00<00:00, 305.15it/s]


Epoch 79/80, Loss: 1.3811


100%|██████████| 196/196 [00:00<00:00, 325.85it/s]

Epoch 80/80, Loss: 1.3810


In [33]:
model.eval()
with torch.no_grad():
    results = []
    for batch in tqdm(test_loader, total=len(test_loader)):

        embeddings = batch['embeddings'].to(DEVICE)


        outputs = model(embeddings.float()) 
        results.append(outputs.cpu().numpy())

    with open("result.txt", "w") as f:
        for val in np.concatenate(results):
            f.write(f"{val[0]}\n")

100%|██████████| 16/16 [00:00<00:00, 72.19it/s]
